### Imports

In [11]:
import pandas as pd
import numpy as np
import sklearn as skit
import math

import geopy
from geopy.distance import VincentyDistance

from scipy import constants
constants.value

<function scipy.constants.codata.value(key)>

###    Funcao  PATH   

In [12]:

#calcula  distancia entre dois pontos B e M
def dist_BS_MS(latB, lonB, latM,lonM):
    coord_base = (latB, lonB)
    coord_mobi = (latM, lonM)
    return geopy.distance.vincenty(coord_base, coord_mobi).m


#calculo do Path loss para dada distancia
def _FSPL(dist, freq):
    return 20*math.log(dist,10) + 20*math.log(freq,10) + 147.55221
    #conferir corretude em: https://www.everythingrf.com/rf-calculators/free-space-path-loss-calculator
#print(_FSPL(2, 1800000000))


In [13]:
#names=["pontoId","lat","lon","pathBTS1","pathBTS2","pathBTS3","pathBTS4","pathBTS5","pathBTS6","taBTS1","taBTS2","taBTS3","taBTS4","taBTS5","taBTS6"]
#BD referencia
df_ref = pd.read_csv("LocTreino_Equipe_6.csv")
df_ref.head()

,pontoId,lat,lon,pathBTS1,pathBTS2,pathBTS3,pathBTS4,pathBTS5,pathBTS6,taBTS1,taBTS2,taBTS3,taBTS4,taBTS5,taBTS6
0,219,-8.076391,-34.895985,120.610000,104.380000,129.970000,123.620000,129.460000,132.080000,1,0,2,0.0,2.0,2
1,798,-8.072939,-34.895676,105.290000,121.523333,106.556667,108.756667,122.556667,126.123333,1,0,2,0.0,1.0,1
2,253,-8.076256,-34.899773,122.256667,110.423333,124.490000,135.256667,142.956667,142.556667,2,1,1,1.0,2.0,2
3,2613,-8.064446,-34.889416,127.423333,132.656667,130.556667,131.856667,126.090000,137.723333,1,2,4,2.0,0.0,1
4,1884,-8.068819,-34.888874,121.490000,136.423333,128.190000,122.723333,107.256667,120.323333,0,1,4,1.0,0.0,1


In [38]:
#Antenas
df_antenas = pd.read_csv("dados_BTSs.csv")
df_antenas

,name,lat,lon,band,bcch,eirp,bts
0,136_2,-8.068361,-34.892722,GSM1800,662,55.59,BTS-1
1,137_3,-8.075917,-34.894611,GSM1800,806,55.59,BTS-2
2,197_1,-8.076361,-34.908000,GSM1800,666,55.59,BTS-3
3,137_1,-8.075917,-34.894611,GSM1800,690,55.59,BTS-4
4,138_2,-8.066000,-34.889444,GSM1800,673,55.59,BTS-5
5,245_2,-8.064583,-34.894583,GSM1800,682,55.59,BTS-6


In [39]:
import folium

recife = folium.Map( location=[-8.05428, -34.8813],    # Coordenadas retiradas do Google Maps
                    zoom_start=12
)

In [40]:
#folium.Marker(location=[loc6['lat'], loc6['lon']]).add_to(recife)
i = 0

while i < len(df_ref):
    aux = df_ref.iloc[i]
    folium.Marker(location=[aux['lat'], aux['lon']]).add_to(recife)
    i = i+1

In [41]:
j = 0

while j < len(df_antenas):
    aux = df_antenas.iloc[j]
    folium.Marker(location=[aux['lat'], aux['lon']],
                  icon=folium.Icon(color='red')).add_to(recife)
    j = j+1

In [42]:
recife

### lat's e lon's das extermidade do mapa
### largura e altura do grid

In [43]:
min_lat = df_ref['lat'].min()
max_lat = df_ref['lat'].max()
min_lon = df_ref['lon'].min()
max_lon = df_ref['lon'].max()

print("latitude maxima: {}".format(max_lat))
print("latitude minima: {}".format(min_lat))
print("***")
print("long maxima: {}".format(max_lon))
print("long minima: {}".format(min_lon))
print("***")
#coord -> (lat lon)
coord_min_min = (min_lat, min_lon)
coord_min_max = (min_lat, max_lon)
coord_max_max = (max_lat, max_lon)
coord_max_min = (max_lat, min_lon)

#rtg -> retangulo

grid_wd = math.ceil(geopy.distance.vincenty(coord_min_min, coord_min_max).m)
grid_ht = math.ceil(geopy.distance.vincenty(coord_min_min, coord_max_min).m)
print ("largura do grid em metros: {}".format(grid_wd))
print ("altura do grid em metros: {}".format(grid_ht))

latitude maxima: -8.059338
latitude minima: -8.077575999999999
***
long maxima: -34.885067
long minima: -34.90683
***
largura do grid em metros: 2399
altura do grid em metros: 2018


### altura, largura e quantidade de celulas
#cell_wd = prop x cell_ht
#cell_wd -> 100 metros

In [44]:
prop = grid_wd/grid_ht
#print(prop)
cell_ht = 50 #em metros
cell_wd = math.ceil(prop*cell_ht)

print("celula tem {} metros de largura".format(cell_wd +1))
print("celula tem {} metros de altura".format(cell_ht))

math.ceil
num_col = math.ceil(grid_wd/cell_wd)
num_row = math.ceil(grid_ht/cell_ht)
n_of_cells = int(num_col*num_row)

print("total de linhas: {}".format(num_row))
print("total de colunas: {}".format(num_col))

print("total de celulas: {}".format(n_of_cells))


celula tem 61 metros de largura
celula tem 50 metros de altura
total de linhas: 41
total de colunas: 40
total de celulas: 1640


### lat's e lon's de todos as celulas do grid

In [45]:

# calculo as coord do centro da primeira celula (mais abaixo e mais a esquerda):
origem = geopy.Point(min_lat, min_lon)
angulo = math.degrees(math.atan(cell_ht/cell_wd))
hipotenusa = math.hypot(cell_ht/2,cell_wd/2)
destination = VincentyDistance(meters=hipotenusa).destination(origem, angulo)
lat0, lon0 = destination.latitude, destination.longitude

#com base em lat0 e lon0 gerar as lats e as lons para as demais celulas:
latx,  lonx = lat0, lon0 
lats, lons = [], []
for l in range(num_row):
    for c in range(num_col):
        #add a lista
        lats.append(latx)
        lons.append(lonx)
        #proximas celulas a direita
        origem = geopy.Point(latx, lonx)
        destination = VincentyDistance(meters=cell_wd).destination(origem, math.degrees(0))
        latx, lonx = destination.latitude, destination.longitude
    #vai para a celula mais a esqueda da linha de cima
    origem = geopy.Point(lat0, lon0)
    destination = VincentyDistance(meters=(l+1)*cell_wd).destination(origem, math.degrees(math.pi/2))
    latx, lonx = destination.latitude, destination.longitude
print(len(lats))
    
data = np.array([lats, lons])
df_cells_coord = pd.DataFrame({'lat':data[0], 'lon':data[1]})
df_cells_coord.head()


1640


,lat,lon
0,-8.077305,-34.906603
1,-8.076762,-34.906603
2,-8.076220,-34.906603
3,-8.075677,-34.906603
4,-8.075135,-34.906603


### Calculo do path de todos as celulas

In [46]:
#def dist_BS_MS(latB, lonB, latM,lonM):
#def _FSPL(dist, freq):

#print (dist_BS_MS(min_lat, min_lon, min_lat, max_lon))
df_antenas.head()

path_loss_BS1 = []
path_loss_BS2 = []
path_loss_BS3 = []
path_loss_BS4 = []
path_loss_BS5 = []
path_loss_BS6 = []

print(len(df_cells_coord))
print(len(path_loss_BS1))
print(len(df_antenas))

n_cells = len(df_cells_coord)

#BS_1
for j in range(n_cells):
    dist = dist_BS_MS(df_antenas.lat[0], df_antenas.lon[0], df_cells_coord.lat[j], df_cells_coord.lon[j])
    pl = _FSPL(dist, 1800*(10**6))
    path_loss_BS1.append(pl)
print(len(path_loss_BS1))

#BS_2
for j in range(n_cells):
    dist = dist_BS_MS(df_antenas.lat[1], df_antenas.lon[1], df_cells_coord.lat[j], df_cells_coord.lon[j])
    pl = _FSPL(dist, 1800*(10**6))
    path_loss_BS2.append(pl)
print(len(path_loss_BS2))

#BS_3
for j in range(n_cells):
    dist = dist_BS_MS(df_antenas.lat[2], df_antenas.lon[2], df_cells_coord.lat[j], df_cells_coord.lon[j])
    pl = _FSPL(dist, 1800*(10**6))
    path_loss_BS3.append(pl)
print(len(path_loss_BS3))

#BS_4
for j in range(n_cells):
    dist = dist_BS_MS(df_antenas.lat[3], df_antenas.lon[3], df_cells_coord.lat[j], df_cells_coord.lon[j])
    pl = _FSPL(dist, 1800*(10**6))
    path_loss_BS4.append(pl)
print(len(path_loss_BS4))

#BS_5
for j in range(n_cells):
    dist = dist_BS_MS(df_antenas.lat[4], df_antenas.lon[4], df_cells_coord.lat[j], df_cells_coord.lon[j])
    pl = _FSPL(dist, 1800*(10**6))
    path_loss_BS5.append(pl)
print(len(path_loss_BS5))

    
#BS_6
for j in range(n_cells):
    dist = dist_BS_MS(df_antenas.lat[5], df_antenas.lon[5], df_cells_coord.lat[j], df_cells_coord.lon[j])
    pl = _FSPL(dist, 1800*(10**6))
    path_loss_BS6.append(pl)
print(len(path_loss_BS6))

print('fim')


1640
0
6
1640
1640
1640
1640
1640
1640
fim


### Data frame com path dos centros das celulas

In [47]:
data_path = np.array([path_loss_BS1, path_loss_BS2, path_loss_BS3, path_loss_BS4, path_loss_BS5, path_loss_BS6])

data_path
df_cells_path = pd.DataFrame({'pathBTS1':data_path[0], 'pathBTS2':data_path[1], 'pathBTS3':data_path[2], 'pathBTS4':data_path[3], 'pathBTS5':data_path[4], 'pathBTS6':data_path[5]})
df_cells_path.head()

,pathBTS1,pathBTS2,pathBTS3,pathBTS4,pathBTS5,pathBTS6
0,397.868141,395.139150,378.047825,395.139150,399.767448,398.380309
1,397.714883,395.102650,376.752229,395.102649,399.641935,398.183793
2,397.566128,395.083763,376.450427,395.083762,399.519134,397.987137
3,397.422548,395.082720,377.344824,395.082720,399.399385,397.790753
4,397.284846,395.099535,378.900643,395.099535,399.283046,397.595109


### Dataframe com coord e path dos centros das celulas

In [48]:
df_cell_coords_paths = df_cells_coord.join(df_cells_path, lsuffix='_df_cells_coord', rsuffix='_df_cells_path')

print(len(df_cell_coords_paths))

df_cell_coords_paths.head()

1640


,lat,lon,pathBTS1,pathBTS2,pathBTS3,pathBTS4,pathBTS5,pathBTS6
0,-8.077305,-34.906603,397.868141,395.139150,378.047825,395.139150,399.767448,398.380309
1,-8.076762,-34.906603,397.714883,395.102650,376.752229,395.102649,399.641935,398.183793
2,-8.076220,-34.906603,397.566128,395.083763,376.450427,395.083762,399.519134,397.987137
3,-8.075677,-34.906603,397.422548,395.082720,377.344824,395.082720,399.399385,397.790753
4,-8.075135,-34.906603,397.284846,395.099535,378.900643,395.099535,399.283046,397.595109


In [49]:
import math

def divideCell(lat, lon):
    pi = math.pi
    half_width = 61/2
    half_length = 50/2
    r_earth = 6378*1000
    
    new_latitude_mais  = lat  + (half_length / r_earth) * (180 / pi);
    new_longitude_mais = lon + (half_width / r_earth) * (180 / pi) / math.cos(lat * pi/180);
    new_latitude_menos  = lat  + ((-half_length) / r_earth) * (180 / pi);
    new_longitude_menos = lon + ((-half_width) / r_earth) * (180 / pi) / math.cos(lat * pi/180);
    
    # (+lat +lon), (+lat -lon), (-lat +lon), (-lat -lon)
    return [(new_latitude_mais, new_longitude_mais), (new_latitude_mais, new_longitude_menos),
            (new_latitude_menos, new_longitude_mais), (new_latitude_menos, new_longitude_menos)]

new_cells = divideCell(-8.077305, -34.906603)
print(new_cells)

[(-8.077080416354999, -34.90632626254915), (-8.077080416354999, -34.90687973745084), (-8.077529583645003, -34.90632626254915), (-8.077529583645003, -34.90687973745084)]


### K-fold cross validation

In [50]:
features = ['pathBTS1', 'pathBTS2', 'pathBTS3', 'pathBTS4', 'pathBTS5', 'pathBTS6']
to_predict=['lat', 'lon']